# PrescientPLMFold
notebook adapted from [huggingface](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_folding.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import torch.nn.functional as F

from lobster.model import PrescientPLMFold

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cuda.matmul.allow_tf32 = True

load the model

In [ ]:
m = PrescientPLMFold(model_name="esmfold_v1")  # pre-trained
# m = PrescientPLMFold(model_name="PPLM")  # random initialization for training from scratch
m.cuda();

In [ ]:
f"{sum(p.numel() for p in m.model.parameters() if p.requires_grad):,}"

In [ ]:
m.model.esm = m.model.esm.half()

In [ ]:
m.model.trunk.set_chunk_size(64)

predict coordinates and convert to pdb

In [ ]:
test_protein = "MGAGASAEEKHSRELEKKLKEDAEKDARTVKLLLLGAGESGKSTIVKQMKIIHQDGYSLEECLEFIAIIYGNTLQSILAIVRAMTTLNIQYGDSARQDDARKLMHMADTIEEGTMPKEMSDIIQRLWKDSGIQACFERASEYQLNDSAGYYLSDLERLVTPGYVPTEQDVLRSRVKTTGIIETQFSFKDLNFRMFDVGGQRSERKKWIHCFEGVTCIIFIAALSAYDMVLVEDDEVNRMHESLHLFNSICNHRYFATTSIVLFLNKKDVFFEKIKKAHLSICFPDYDGPNTYEDAGNYIKVQFLELNMRRDVKEIYSHMTCATDTQNVKFVFDAVTDIIIKENLKDCGLF"

tokenized_input = m.tokenizer([test_protein], return_tensors="pt", add_special_tokens=False)['input_ids']

In [ ]:
tokenized_input = tokenized_input.cuda()

In [ ]:
with torch.no_grad():
    output = m.model(tokenized_input)

In [ ]:
output.keys()

In [ ]:
m.model.config

In [ ]:
# get hidden states from folding trunk
output.states.shape  # (num structure blocks, B, L, sequence hidden dim)

In [ ]:
output.frames.shape

In [ ]:
output.positions.shape  # (..., B, L, 14 (atom14), 3 (xyz))

In [ ]:
output.plddt.shape  # (B, L, 37 (atom37))

In [ ]:
pdb_file = m.model.output_to_pdb(output)[0]

In [ ]:
with open('output.pdb', 'w') as f:
    f.write(pdb_file)

predict coordinates of a homodimer.

we insert a "linker" of flexible glycine residues between each chain we want to fold simultaneously, and then we offset the position IDs for each chain from each other, so that the model treats them as being very distant portions of the same long chain  

Tip: If you're trying to predict a multimeric structure and you're getting low-quality outputs, try varying the order of the chains (if it's a heteropolymer) or the length of the linker

In [ ]:
sequence = "MRLIPLHNVDQVAKWSARYIVDRINQFQPTEARPFVLGLPTGGTPLKTYEALIELYKAGEVSFKHVVTFNMDEYVGLPKEHPESYHSFMYKNFFDHVDIQEKNINILNGNTEDHDAECQRYEEKIKSYGKIHLFMGGVGVDGHIAFNEPASSLSSRTRIKTLTEDTLIANSRFFDNDVNKVPKYALTIGVGTLLDAEEVMILVTGYNKAQALQAAVEGSINHLWTVTALQMHRRAIIVCDEPATQELKVKTVKYFTELEASAIRSVK"

linker = 'G' * 25

homodimer_sequence = sequence + linker + sequence

In [ ]:
tokenized_homodimer = m.tokenizer([homodimer_sequence], return_tensors="pt", add_special_tokens=False)

In [ ]:
# add a large offset to the position IDs of the second chain
with torch.no_grad():
    position_ids = torch.arange(len(homodimer_sequence), dtype=torch.long)
    position_ids[len(sequence) + len(linker):] += 512

In [ ]:
tokenized_homodimer['position_ids'] = position_ids.unsqueeze(0)

tokenized_homodimer = {key: tensor.cuda() for key, tensor in tokenized_homodimer.items()}

In [ ]:
with torch.no_grad():
    output = m.model(**tokenized_homodimer)

In [ ]:
# remove the poly-G linker from the output, so we can display the structure as fully independent chains
linker_mask = torch.tensor([1] * len(sequence) + [0] * len(linker) + [1] * len(sequence))[None, :, None]

output['atom37_atom_exists'] = output['atom37_atom_exists'] * linker_mask.to(output['atom37_atom_exists'].device)

In [ ]:
pdb_file = m.model.output_to_pdb(output)[0]

In [ ]:
with open('homodimer.pdb', 'w') as f:
    f.write(pdb_file)

## data stuff

In [ ]:
from lobster.data import FastaLightningDataModule
from lobster.tokenization import PmlmTokenizer
import importlib.resources

In [ ]:
path = importlib.resources.files("lobster") / "assets" / '3di_tokenizer'

t = PmlmTokenizer.from_pretrained(path)
inputs = ['GdPfQaPfIlSvRvLvEcQvClGpId']

In [ ]:
tokenized_inputs = t(inputs)

In [ ]:
tokenized_inputs['input_ids']

In [ ]:
out = t.decode(token_ids=tokenized_inputs['input_ids'][0], skip_special_tokens=True).replace(' ', '')

In [ ]:
assert inputs[0] == out

In [ ]:
dm = FastaLightningDataModule(path_to_fasta=[
    "/scratch/site/u/freyn6/data/fasta/pdb_3di.fasta",
],
                              tokenizer_dir='3di_tokenizer',
                             batch_size=4,
                             )
dm.setup(stage='fit')

In [ ]:
batch = next(iter(dm.train_dataloader()))

In [ ]:
dm._transform_fn._auto_tokenizer

In [ ]:
batch['input_ids'][0]